In [29]:
!pip install noisereduce

In [35]:
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub
import noisereduce as nr
import json
from collections import defaultdict
import os

class InstrumentDetector:
    def __init__(self):
        self.model = hub.load('https://tfhub.dev/google/yamnet/1')
        self.class_names = self._get_class_names()
        self.instrument_mapping = {
            'guitar': ['guitar', 'acoustic guitar', 'electric guitar'],
            'piano': ['piano', 'keyboard'],
            'drums': ['drum', 'drum kit', 'snare drum'],
            'bass': ['bass', 'bass guitar'],
            'violin': ['violin', 'fiddle'],
            'vocals': ['singing', 'voice'],
            'synthesizer': ['synthesizer'],
            'brass': ['trumpet', 'trombone'],
            'woodwind': ['flute', 'clarinet']
        }

    def _get_class_names(self):
        class_map_path = self.model.class_map_path().numpy().decode('utf-8')
        with tf.io.gfile.GFile(class_map_path) as f:
            return [line.strip().split(',')[2] for line in f]

    def _preprocess_audio(self, file_path):
        audio, sr = librosa.load(file_path, sr=16000)
        audio = audio / np.max(np.abs(audio))
        return audio

    def _analyze_audio(self, audio):
        scores, _, _ = self.model(audio)
        mean_scores = np.mean(scores, axis=0)
        return mean_scores

    def _map_to_instruments(self, scores):
        instrument_scores = defaultdict(float)
        total_score = 0

        for class_idx, score in enumerate(scores):
            class_name = self.class_names[class_idx].lower()
            for instrument, keywords in self.instrument_mapping.items():
                if any(keyword in class_name for keyword in keywords):
                    instrument_scores[instrument] += float(score)  # Convert to native float
                    total_score += float(score)
                    break

        if total_score > 0:
            return {inst: round((score/total_score)*100, 2) for inst, score in instrument_scores.items()}
        return {}

    def detect_instruments(self, file_path):
        try:
            if not os.path.exists(file_path):
                return {
                    "status": "error",
                    "message": "File not found",
                    "input_file": file_path
                }

            audio = self._preprocess_audio(file_path)
            scores = self._analyze_audio(audio)
            results = self._map_to_instruments(scores)

            # Convert numpy values to native Python types
            converted_results = {k: float(v) for k, v in results.items()}

            return {
                "status": "success",
                "input_file": os.path.basename(file_path),
                "analysis": converted_results
            }
        except Exception as e:
            return {
                "status": "error",
                "message": str(e),
                "input_file": os.path.basename(file_path) if file_path else "unknown"
            }

def main():
    print("Music Instrument Detection System")
    print("--------------------------------")

    # Get user input
    while True:
        file_path = input("Enter path to audio file (or 'quit' to exit): ").strip()

        if file_path.lower() == 'quit':
            break

        # Initialize detector and process file
        detector = InstrumentDetector()
        result = detector.detect_instruments(file_path)

        # Output JSON result
        print("\nAnalysis Results:")
        print(json.dumps(result, indent=2))
        print("\n" + "="*50 + "\n")

if __name__ == "__main__":
    main()

Music Instrument Detection System
--------------------------------
Enter path to audio file (or 'quit' to exit): /content/song.mp3

Analysis Results:
{
  "status": "success",
  "input_file": "song.mp3",
  "analysis": {
    "synthesizer": 11.04,
    "vocals": 55.88,
    "guitar": 27.49,
    "piano": 0.81,
    "drums": 3.84,
    "brass": 0.59,
    "violin": 0.29,
    "bass": 0.04,
    "woodwind": 0.02
  }
}


Enter path to audio file (or 'quit' to exit): exit

Analysis Results:
{
  "status": "error",
  "message": "File not found",
  "input_file": "exit"
}


Enter path to audio file (or 'quit' to exit): quit


In [36]:
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub
import noisereduce as nr
import json
from collections import defaultdict
import os

class InstrumentDetector:
    def __init__(self):
        self.model = hub.load('https://tfhub.dev/google/yamnet/1')
        self.class_names = self._get_class_names()
        self.instrument_mapping = {
            'guitar': ['guitar', 'acoustic guitar', 'electric guitar'],
            'piano': ['piano', 'keyboard'],
            'drums': ['drum', 'drum kit', 'snare drum'],
            'bass': ['bass', 'bass guitar'],
            'violin': ['violin', 'fiddle'],
            'vocals': ['singing', 'voice'],
            'synthesizer': ['synthesizer'],
            'brass': ['trumpet', 'trombone'],
            'woodwind': ['flute', 'clarinet']
        }

    def _get_class_names(self):
        class_map_path = self.model.class_map_path().numpy().decode('utf-8')
        with tf.io.gfile.GFile(class_map_path) as f:
            return [line.strip().split(',')[2] for line in f]

    def _preprocess_audio(self, file_path):
        audio, sr = librosa.load(file_path, sr=16000)
        audio = audio / np.max(np.abs(audio))
        return audio

    def _analyze_audio(self, audio):
        scores, _, _ = self.model(audio)
        mean_scores = np.mean(scores, axis=0)
        return mean_scores

    def _map_to_instruments(self, scores):
        instrument_scores = defaultdict(float)
        total_score = 0

        for class_idx, score in enumerate(scores):
            class_name = self.class_names[class_idx].lower()
            for instrument, keywords in self.instrument_mapping.items():
                if any(keyword in class_name for keyword in keywords):
                    instrument_scores[instrument] += float(score)
                    total_score += float(score)
                    break

        if total_score > 0:
            return {inst: round((score/total_score)*100, 2) for inst, score in instrument_scores.items()}
        return {}

    def detect_instruments(self, file_path):
        try:
            if not os.path.exists(file_path):
                return {
                    "status": "error",
                    "message": "File not found",
                    "input_file": file_path
                }

            audio = self._preprocess_audio(file_path)
            scores = self._analyze_audio(audio)
            results = self._map_to_instruments(scores)


            converted_results = {k: float(v) for k, v in results.items()}

            # O/P dictionary
            output = {
                "status": "success",
                "input_file": os.path.basename(file_path),
                "analysis": converted_results
            }

            # JSON save in local
            output_filename = os.path.splitext(os.path.basename(file_path))[0] + '.json'
            with open(output_filename, 'w') as f:
                json.dump(output, f, indent=2)

            return output

        except Exception as e:
            return {
                "status": "error",
                "message": str(e),
                "input_file": os.path.basename(file_path) if file_path else "unknown"
            }

def main():
    print("Music Instrument Detection System")
    print("--------------------------------")
    print("Results will be saved as [songname].json in current directory\n")

    # Get user input
    while True:
        file_path = input("Enter path to audio file (or 'quit' to exit): ").strip()

        if file_path.lower() == 'quit':
            break

        # Initialize detector and process file
        detector = InstrumentDetector()
        result = detector.detect_instruments(file_path)

        # Output results to console
        print("\nAnalysis Results:")
        print(json.dumps(result, indent=2))

        if result['status'] == 'success':
            output_filename = os.path.splitext(os.path.basename(file_path))[0] + '.json'
            print(f"\nResults saved to: {output_filename}")
        print("\n" + "="*50 + "\n")

if __name__ == "__main__":
    main()

Music Instrument Detection System
--------------------------------
Results will be saved as [songname].json in current directory

Enter path to audio file (or 'quit' to exit): /content/song.mp3

Analysis Results:
{
  "status": "success",
  "input_file": "song.mp3",
  "analysis": {
    "synthesizer": 11.04,
    "vocals": 55.88,
    "guitar": 27.49,
    "piano": 0.81,
    "drums": 3.84,
    "brass": 0.59,
    "violin": 0.29,
    "bass": 0.04,
    "woodwind": 0.02
  }
}

Results saved to: song.json


Enter path to audio file (or 'quit' to exit): quit
